In [2]:
%%capture
# Installs Unsloth, Xformers (Flash Attention) and all other packages!
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install --no-deps "xformers<0.0.27" "trl<0.9.0" peft accelerate bitsandbytes
!pip install rouge_score

In [6]:
import torch
from unsloth import FastLanguageModel
from transformers import TextStreamer
from transformers import AutoModelForCausalLM, AutoTokenizer

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


In [7]:
MODEL_NAME =  "sbtraining2020/esubjectgen_llama31_clean"

In [8]:
max_seq_length = 2048
dtype = None
load_in_4bit = True

In [10]:
from transformers import AutoTokenizer, AutoModelForCausalLM
hf_tokenizer = AutoTokenizer.from_pretrained("sbtraining2020/esubjectgen_llama31_clean")
hf_model = AutoModelForCausalLM.from_pretrained("sbtraining2020/esubjectgen_llama31_clean")

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [13]:
alpaca_prompt = """Below is a instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [14]:
def get_output(email_text):
  #"Generate a summary for the Input section in not more than 50 words"
  inputs = hf_tokenizer(
                        [
                            alpaca_prompt.format(
                                # "Generate a subject for the email body defined in Input section in not more than 50 words", # instruction
                                "Generate a subject for the email text in not more than 10 words",
                                email_text, # input
                                "", # output - leave this blank for generation!
                            )
                        ], return_tensors = "pt").to("cuda")

  from transformers import TextStreamer
  text_streamer = TextStreamer(hf_tokenizer)
  result = hf_model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
  decoded = hf_tokenizer.batch_decode(result)
  response_text = decoded[0].split("### Response:")[-1].strip().replace('<|end_of_text|>','').replace('<|begin_of_text|>:// ','')
  return response_text


In [15]:
email_text = """Plove is going to go to Dallas.
We are going to leave next Friday when he  gets done (7ish) and go up for the game.
The game is at 11 in the morning,  so we will come home directly after it.
Plove says he has a friend who has a  place in Dallas that we can crash at if we don't want to pay for a hotel.
Do you want to go?
        """
result = get_output(email_text)


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is a instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Generate a subject for the email text in not more than 10 words

### Input:
Plove is going to go to Dallas.
We are going to leave next Friday when he  gets done (7ish) and go up for the game.
The game is at 11 in the morning,  so we will come home directly after it.
Plove says he has a friend who has a  place in Dallas that we can crash at if we don't want to pay for a hotel.
Do you want to go?
        

### Response:
Dallas<|end_of_text|>


In [16]:
result

'Dallas'